<a href="https://colab.research.google.com/github/beknur/NLP/blob/master/HypernymTitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Lab 7 
#### Rouge Metrics

In [32]:
pip install fuzzywuzzy

In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "data.csv"

In [41]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
df = pd.read_csv(__PATH__,sep=";",header=0)
df

,id,updatedDate,publishedDate,title,summary,authors,category,metaData,downloadLink,filePath
0,http://arxiv.org/abs/1407.6950v1,2014-07-24T16:56:39Z,2014-07-24T16:56:39Z,"How,whenAndHowMuchACardDeckIsWellShuffled.pdf",The Thesis Consider The Mixing Of Few 3 4 ...,Benjamin Isac Fargion,cs.DM,"Italian Thesis In Engeenering Computer, 26 Feb...",http://arxiv.org/pdf/1407.6950v1.pdf,"./files/How,whenAndHowMuchACardDeckIsWellShuff..."
1,http://arxiv.org/abs/0907.0618v1,2009-07-03T12:35:10Z,2009-07-03T12:35:10Z,QuantumIsometryGroups.pdf,This Thesis Contains The Formulation And Com...,Jyotishman Bhowmick,math.OA,Thesis,http://arxiv.org/pdf/0907.0618v1.pdf,./files/QuantumIsometryGroups.pdf
2,http://arxiv.org/abs/1806.09601v2,2018-07-14T17:06:27Z,2018-06-25T17:55:59Z,ComputationAndBoundingOfFolkmanNumbers.pdf,Phd Thesis Under The Supervision Of Professo...,Aleksandar Bikov,math.CO,PhD Thesis,http://arxiv.org/pdf/1806.09601v2.pdf,./files/ComputationAndBoundingOfFolkmanNumbers...
3,http://arxiv.org/abs/1905.03014v1,2019-05-08T11:47:34Z,2019-05-08T11:47:34Z,OnChurch'sThesisInCubicalAssemblies.pdf,"We Show That Church's Thesis, The Axiom Stat...","Andrew Swan, Taichi Uemura,",math.LO,0,http://arxiv.org/pdf/1905.03014v1.pdf,./files/OnChurch'sThesisInCubicalAssemblies.pdf
4,http://arxiv.org/abs/1901.04911v1,2019-01-15T16:24:07Z,2019-01-15T16:24:07Z,UnconstrainedChurchTuringThesisCannotPossiblyB...,The Church Turing Thesis Asserts That If A P...,Yuri Gurevich,cs.LO,0,http://arxiv.org/pdf/1901.04911v1.pdf,./files/UnconstrainedChurchTuringThesisCannotP...
...,...,...,...,...,...,...,...,...,...,...
991,http://arxiv.org/abs/0709.0497v1,2007-09-04T18:40:50Z,2007-09-04T18:40:50Z,HardSpectatorInteractionsInBToPiPiAtOrderAlpha...,In The Present Thesis I Discuss The Hard Spe...,Volker Pilipp,hep-ph,PhD thesis,http://arxiv.org/pdf/0709.0497v1.pdf,./files/HardSpectatorInteractionsInBToPiPiAtOr...
992,http://arxiv.org/abs/0709.2936v1,2007-09-18T23:56:17Z,2007-09-18T23:56:17Z,BayesianClassificationAndRegressionWithHighDim...,This Thesis Responds To The Challenges Of Us...,Longhai Li,stat.ML,"PhD Thesis Submitted to University of Toronto,...",http://arxiv.org/pdf/0709.2936v1.pdf,./files/BayesianClassificationAndRegressionWit...
993,http://arxiv.org/abs/0710.1790v1,2007-10-09T15:20:08Z,2007-10-09T15:20:08Z,StaticAndDynamicPropertiesOfHadronicSystemsWit...,This Thesis Has Been Devoted To The Study Of...,J. M. Verde-Velasco,hep-ph,"PhD thesis, Supervisors: E. Hernandez, J. Niev...",http://arxiv.org/pdf/0710.1790v1.pdf,./files/StaticAndDynamicPropertiesOfHadronicSy...
994,http://arxiv.org/abs/0711.0873v2,2007-11-11T19:21:04Z,2007-11-06T14:13:50Z,AConformalApproachToNumericalCalculationsOfAsy...,This Thesis Is Concerned With The Developmen...,Anıl Zenginoğlu,gr-qc,"PhD thesis, Max-Planck Institute for Gravitati...",http://arxiv.org/pdf/0711.0873v2.pdf,./files/AConformalApproachToNumericalCalculati...


#### Preprocessing the title to list of tokens

In [0]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

In [0]:
res = {}
for title in titles:
    synsets = {}
    for word in title:
        synsets[word]=[synset for synset in wn.synsets(word)]
    res[title] = synsets

In [0]:
def get_hyperonyms(token):
    hyperonyms = {}
    for word in token:
        hyperonyms_ = []
        for synset in wn.synsets(word):
            hyperonyms_.extend(synset.hypernyms())
        hyperonyms[word] = set(hyperonyms_)
    return hyperonyms

def get_fscore_and_intr(a, b):
    intr = a.intersection(b)
    if len(intr) == 0:
        return 0, intr
    
    recall = len(intr) / len(a)
    prec = len(intr) / len(b)
    score = 2*recall*prec/(recall+prec)
    return score, intr

def distance(a,b):
    ### Put your code here
    ### В переменной synsets помимо нормализованного заголовка хранятся синсеты для каждого токена из заголовка.
    ### у синсетов есть гиперонимы про то как из брать здесь
    ### Если у токенов заголовков есть общие гиперонимы тогда заголовки связаны несмотря на то что слова разные
    a = set(a)
    b = set(b)
    
    fscore, intr = get_f1_and_intr(a, b)
    
    a = a - intr
    b = b - intr
    
    if len(a) == 0 or len(b) == 0:
        return 1.0 - fscore
    
    buff_a = get_hyperonyms(a)
    buff_b = get_hyperonyms(b)
    
    hyperonym_p = 0
    hyperonym_2nd = set()
    for word_a in a:
        for word_b in b:
            common_hyps = buff_a[word_a].intersection(buff_b[word_b])
            if len(common_hyps) > 0:
                hyperonym_p += 1
                strs = []
                for s in common_hyps:
                    strs.extend(s.lemma_names())
                hyperonym_2nd.update(strs)
    
    if len(hyperonym_2nd) > 0:
        buff_hyperonym_2nd = get_hyperonyms(hyperonym_2nd)
        for h in hyperonym_2nd:
            for word_a in a:
                a_common_hyps = buff_hyperonym_2nd[h].intersection(buff_a[word_a])
                if len(a_common_hyps) > 0:
                    hyperonym_p += 0.5
            
            for word_b in b:
                hyperonym_com_b = buff_hyperonym_2nd[h].intersection(buff_b[word_b])
                if len(hyperonym_com_b) > 0:
                    hyperonym_p += 0.5
                
    fscore_hyp = 2*hyperonym_p/len(a)*hyperonym_p/len(b)/(hyperonym_p/len(a)+hyperonym_p/len(b)) if hyperonym_p > 0 else 0
    f_res = (2*fscore+fscore_hyp)/3
    return (1.0 - f_res)

In [0]:
buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        dist[lli,rri]=distance(ll[0],rr[0])

#### Top ten closest articles with fuzzy metrics of titles

In [25]:
@interact(ind=(0,len(buff)-1,1))
def h(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint([buff[i][0] for i in dist[ind][:].argsort()[1:11]])


interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…

In [26]:
@interact(ind=(0,len(buff)-1,1))
def hypernyms(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint(buff[ind][1])

interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…